In [ ]:
from collections import OrderedDict
from fenics import *
import matplotlib.pyplot as plt
import numpy as np
from fenics_adjoint import *
import moola
from mshr import *
from DarcySolver import solve_darcy
from BiotSolver import solve_biot
from PlottingHelper import (plot_pressures_cross_section, 
                            plot_pressures_timeslice,
                            extract_cross_section, style_dict)

# time stepping
T = 1.2           # final time
num_steps = 12    # number of time steps
dt = T/ num_steps
times = np.linspace(dt, T, num_steps)

# material parameter
kappa = 1e-17       # permeability
visc = 0.8*1e-3     # viscocity 
K = kappa/visc      # hydraulic conductivity
c = 2*1e-4  # storage coefficent
alpha = 1.0  # Biot-Willis coefficient


# Biot material parameters
E = 1500.0          # Young modulus
nu = 0.479         # Poisson ratio

material_parameter = dict()
material_parameter["c"] = c
material_parameter["K"] = K
material_parameter["lmbda"] = nu*E/((1.0-2.0*nu)*(1.0+nu)) 
material_parameter["mu"] = E/(2.0*(1.0+nu))
material_parameter["alpha"] = alpha
mmHg2Pa = 132.32

# create mesh and mark boundaries
N = 10 # resolution
brain_radius = 0.1 
ventricle_radius = brain_radius/3
brain = Circle(Point(0,0), brain_radius)
ventricle = Circle(Point(0,0), ventricle_radius)
brain = brain - ventricle
mesh = Mesh(generate_mesh(brain, N))

ventricle = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] < R*R*0.95)",
                              R =brain_radius )
skull = CompiledSubDomain("on_boundary && (x[0]*x[0] + x[1]*x[1] >= R*R*0.95 )",
                          R = brain_radius)
boundary_marker = MeshFunction("size_t", mesh, mesh.topology().dim()-1, value=0)
skull.mark(boundary_marker, 1)
ventricle.mark(boundary_marker, 2)
x_coords = np.linspace(ventricle_radius, brain_radius, 20)
slice_points = [Point(x, 0.0) for x in x_coords]

# set analytical expressions
f = 1
A = 1/(brain_radius - ventricle_radius)*mmHg2Pa
p_obs = Expression("A*(sqrt(x[0]*x[0] + x[1]*x[1]) - R_vent)*sin(2*pi*f*t)",
                    A=A, f=f, t=0, R_vent=ventricle_radius, degree=2)

f_ana = Expression("- K*A/(sqrt(x[0]*x[0] + x[1]*x[1]))*sin(2*pi*f*t)" +
                   "+ c*2*A*pi*f*(sqrt(x[0]*x[0] + x[1]*x[1]) - R_vent)*cos(2*pi*f*t)",
                    K=K, A=A, c=c, f=f, t=0, R_vent=ventricle_radius, degree=2)


In [ ]:
# compute darcy flow
boundary_conditions_p = {1:{"Dirichlet":p_obs},
                         2:{"Dirichlet":p_obs}}
darcy_solution = solve_darcy(mesh, f_ana, T, num_steps, K,
                             boundary_marker, boundary_conditions_p,
                             c=c, p_initial=p_obs)
darcy_solution = [s.copy() for s in darcy_solution]

In [ ]:
# compute biot flow
f = Constant((0.0,0.0))
n = FacetNormal(mesh)
boundary_conditions_u = {1:{"Dirichlet":Constant((0.0, 0.0))},
                         2:{"Neumann": -p_obs*n}
                        }


biot_solution = solve_biot(mesh, f, f_ana, T, num_steps, material_parameter,
                           boundary_marker, boundary_conditions_p,
                           boundary_marker, boundary_conditions_u)
biot_solution = [s.copy() for s in biot_solution]
biot_pressure_solution = [s.split()[2] for s in biot_solution]

In [ ]:
darcy_slice = extract_cross_section(darcy_solution, slice_points)/mmHg2Pa
biot_slice = extract_cross_section(biot_pressure_solution, slice_points)/mmHg2Pa

pressures = {"biot":biot_slice,
             "darcy":darcy_slice}
style_dict["biot"]= {"marker":"*"}

In [ ]:
for i in [1,5,10]:
    plot_pressures_cross_section(pressures, i, x_coords)
    plt.title(f"t={times[i]:.3f} s")

In [ ]:
for i in [2,6,12,16]:
    plot_pressures_timeslice(pressures, i, times)
    plt.title(f"Point: ({slice_points[i].x():.3f}, {slice_points[i].y():.3f})")

In [ ]:
p = biot_pressure_solution[0]
plot(p)

In [ ]:
assemble(div(grad(p))*dx)